# Computational Statistics Assignment 1

#### Tianqi Wang

In [22]:
%matplotlib inline
from IPython.core.pylabtools import figsize
import numpy as np
import pymc3 as pm
import theano.tensor as tt
import matplotlib.pyplot as plt

## Question 1

In [11]:
count_data = np.loadtxt("Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/Chapter1_Introduction/data/txtdata.csv")
n_count_data = len(count_data)

In [13]:
len(count_data)

74

In [41]:
with pm.Model() as model:
    alpha1 = 40/count_data[:40].sum()  # Recall count_data is the
    alpha2 = (n_count_data-40)/count_data[40:].sum()  # variable that holds our txt counts
    lambda_1 = pm.Exponential("lambda_1", alpha1)
    lambda_2 = pm.Exponential("lambda_2", alpha2)
    tau = pm.Deterministic("tau", 40)
#     tau = pm.DiscreteUniform("tau", lower=0, upper=n_count_data - 1)

AttributeError: 'int' object has no attribute 'copy'

In [37]:
with model:
    idx = np.arange(n_count_data) # Index
    lambda_ = pm.math.switch(tau > idx, lambda_1, lambda_2)

In [38]:
with model:
    observation = pm.Poisson("obs", lambda_, observed=count_data)

In [39]:
### Mysterious code to be explained in Chapter 3.
with model:
    step = pm.Metropolis()
    trace = pm.sample(10000, tune=5000,step=step)

Multiprocess sampling (2 chains in 2 jobs)
CompoundStep
>Metropolis: [lambda_2]
>Metropolis: [lambda_1]
Sampling 2 chains: 100%|██████████| 30000/30000 [00:05<00:00, 5347.88draws/s]
The number of effective samples is smaller than 25% for some parameters.


In [40]:
lambda_1_samples = trace['lambda_1']
lambda_2_samples = trace['lambda_2']
tau_samples = trace['tau']

KeyError: 'Unknown variable tau'

In [ ]:
figsize(12.5, 10)
#histogram of the samples:

ax = plt.subplot(311)
ax.set_autoscaley_on(False)

plt.hist(lambda_1_samples, histtype='stepfilled', bins=30, alpha=0.85,
         label="posterior of $\lambda_1$", color="#A60628", normed=True)
plt.legend(loc="upper left")
plt.title(r"""Posterior distributions of the variables
    $\lambda_1,\;\lambda_2,\;\tau$""")
plt.xlim([15, 30])
plt.xlabel("$\lambda_1$ value")

ax = plt.subplot(312)
ax.set_autoscaley_on(False)
plt.hist(lambda_2_samples, histtype='stepfilled', bins=30, alpha=0.85,
         label="posterior of $\lambda_2$", color="#7A68A6", normed=True)
plt.legend(loc="upper left")
plt.xlim([15, 30])
plt.xlabel("$\lambda_2$ value")

plt.subplot(313)
w = 1.0 / tau_samples.shape[0] * np.ones_like(tau_samples)
plt.hist(tau_samples, bins=n_count_data, alpha=1,
         label=r"posterior of $\tau$",
         color="#467821", weights=w, rwidth=2.)
plt.xticks(np.arange(n_count_data))

plt.legend(loc="upper left")
plt.ylim([0, .75])
plt.xlim([35, len(count_data)-20])
plt.xlabel(r"$\tau$ (in days)")
plt.ylabel("probability");

In [ ]:
with pm.Model() as model:
    alpha1 = 40/count_data[:40].sum()  # Recall count_data is the
    alpha2 = (n_count_data-40)/count_data[40:].sum()  # variable that holds our txt counts
    lambda_1 = pm.Exponential("lambda_1", alpha1)
    lambda_2 = pm.Exponential("lambda_2", alpha2)
    tau = pm.DiscreteUniform("tau", lower=0, upper=n_count_data - 1)
#     tau = pm.DiscreteUniform("tau", lower=0, upper=n_count_data - 1)
with model:
    idx = np.arange(n_count_data) # Index
    lambda_ = pm.math.switch(tau > idx, lambda_1, lambda_2)
with model:
    observation = pm.Poisson("obs", lambda_, observed=count_data)
### Mysterious code to be explained in Chapter 3.
with model:
    step = pm.Metropolis()
    trace = pm.sample(10000, tune=5000,step=step)
lambda_1_samples = trace['lambda_1']
lambda_2_samples = trace['lambda_2']
tau_samples = trace['tau']